In [1]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import glob
import pandas as pd
from openslide import OpenSlide
from torch.utils.data import Dataset
from PIL import Image, ImageOps, ImageEnhance 

label_csv = pd.read_csv('/home/tsurendr/Test_labels.csv')
# print(label_csv)
list_wsi = label_csv['filenames'].tolist()
# print(list_wsi)
class MyDaatset(Dataset):
    #pass main directory to all other patient folders
    def __init__(self, path,transform):
        # testing_set = []
        # print(path)
        self.files = path
        # print(self.files)
        # print(len(self.files))
        # print(type(self.files))
        self.transform = transform
        # self.labels = labels_wsi
        self.data = []
        for i in list_wsi:
            # print(i)
            new_path = self.files+i+'_tiles/*'
            # print(new_path)
            all_images = glob.glob(new_path)
            for patch in all_images:
                # print(label_csv[label_csv["filenames"] == (list_wsi[i])]['labels'])
                label = label_csv[label_csv["filenames"] == (i)]['labels'].iloc[0]
                # print(label)
                # name = patch
                # file = Image.open(patch)
                # file = self.transform(file)
                label_conv = 0
                if label.startswith("H-CKD"):
                    label_conv=2
                elif label.startswith("Healthy"):
                    label_conv=1
                elif label.startswith("DKD"):
                    label_conv=0
                # print(all_images)
                self.data.append([patch,label_conv])
        # print(self.data)
    
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        img, label_conv = self.data[idx]
        name = img
        image = Image.open(img)
        image = self.transform(image)
        return image,label_conv,name
        


PATH = "binary_label_classifier_patient_level.pt"
data_transforms = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

In [2]:
test_set = MyDaatset('/data1/DKDimages/All_patch_Folders/', transform = data_transforms)
# print(testing_set)
tester = torch.utils.data.DataLoader(test_set, batch_size=4,
                                             shuffle=True, num_workers=8)

print("done")


done


In [3]:
print(len(test_set))
print(len(tester))
print(type(test_set))
print(type(tester))
path_sum = 0
for i, data in enumerate(tester):
    images,labels,paths = data
    # print(data)
    if paths[0]: 
        path_sum+=1
        print("path 0", paths[0], labels[0])
    if paths[1]:
        path_sum+=1
        print("path 1", paths[1], labels[1])
        
    try:
        if paths[2]:
            path_sum+=1
            print("path 2", paths[2], labels[2])
            
    except IndexError:
        pass
    
    try:
        if paths[3]:
            path_sum+=1
            print("path 3", paths[3], labels[3])
            
    except IndexError:
        pass
print("done")
print(path_sum)

5531
1383
<class '__main__.MyDaatset'>
<class 'torch.utils.data.dataloader.DataLoader'>
path 0 /data1/DKDimages/All_patch_Folders/829067a5-eda4-415d-8e2e-fa9a15ac583c_18-162_PAS_2of6_tiles/829067a5-eda4-415d-8e2e-fa9a15ac583c_18-162_PAS_2of6_1678.png tensor(0)
path 1 /data1/DKDimages/All_patch_Folders/dbdcd9a2-62a7-4988-86a2-90cf1ea57242_S-1910-000042_PAS_2of2_tiles/dbdcd9a2-62a7-4988-86a2-90cf1ea57242_S-1910-000042_PAS_2of2_06537.png tensor(2)
path 2 /data1/DKDimages/All_patch_Folders/502d2911-4815-44e0-b66f-b1910e8808e9_S-1910-000043_PAS_1of2_tiles/502d2911-4815-44e0-b66f-b1910e8808e9_S-1910-000043_PAS_1of2_13515.png tensor(2)
path 3 /data1/DKDimages/All_patch_Folders/829067a5-eda4-415d-8e2e-fa9a15ac583c_18-162_PAS_2of6_tiles/829067a5-eda4-415d-8e2e-fa9a15ac583c_18-162_PAS_2of6_1516.png tensor(0)
path 0 /data1/DKDimages/All_patch_Folders/e6c166fd-0b8b-4dbe-8fea-7fd3310d9243_S-2001-005356_PAS_2of2_tiles/e6c166fd-0b8b-4dbe-8fea-7fd3310d9243_S-2001-005356_PAS_2of2_05443.png tensor(0)
pa

In [ ]:


plt.ion()   # interactive mode
data_transforms = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

temp_DKD = glob.glob('/data1/DKDimages/KPMP_WSI_NEW_LABELS/Has_DKD/*')
wsi_HasDKD = []
wsi_all = []
for x in temp_DKD:
    
    dkd_split = x.split("/")[-1].split(".")[0].split('_')[0]
#     print(dkd_split)
    wsi_HasDKD.append(dkd_split)
    wsi_all.append(dkd_split)
# print(wsi_HasDKD)

temp_Healthy = glob.glob('/data1/DKDimages/KPMP_WSI_NEW_LABELS/Healthy/*')
wsi_Healthy = []
for x in temp_Healthy:
    
    dkd_split = x.split("/")[-1].split(".")[0].split('_')[0]
#     print(dkd_split)
    wsi_Healthy.append(dkd_split)
    wsi_all.append(dkd_split)
# print(wsi_Healthy)

# print(wsi_all)
wsi_dict = dict()
for i in wsi_all:
    wsi_dict[i] = dict()
    wsi_dict[i]["Healthy"] = 0
    wsi_dict[i]["Has_DKD"] = 0

# print(wsi_dict)

params = {'batch_size': 4,
          'shuffle': True,
          'num_workers': 8}

class_names = ('Has_DKD', 'Healthy')
# data_dir = '/projectnb/vkolagrp/Tsurendr/Deepslide-NewLabels/train_folder/'
# print(os.path.exists(data_dir))

# training_set = Dataset((os.path.join(data_dir, 'train')), labels)
# training_generator = torch.utils.data.DataLoader(training_set, **params)

# validation_set = Dataset(os.path.join(data_dir, 'val'), labels)
# validation_generator = torch.utils.data.DataLoader(validation_set, **params)



# image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
#                                           data_transforms[x])
#                   for x in ['train', 'val']}

image_datasets_train = ImageFolderWithPaths('/data1/DKDimages/Binary Classifier_patches/train_folder/train', transform = data_transforms)
image_datasets_val = ImageFolderWithPaths('/data1/DKDimages/Binary Classifier_patches/train_folder/val', transform = data_transforms)
image_datasets_test = ImageFolderWithPaths('/data1/DKDimages/Binary Classifier_patches/new_test_patches/train', transform = data_transforms)

print("validation set length: ", len(image_datasets_val))
dataloader_train = torch.utils.data.DataLoader(image_datasets_train, batch_size=4,
                                             shuffle=True, num_workers=8)
dataloader_val = torch.utils.data.DataLoader(image_datasets_val, batch_size=4,
                                             shuffle=True, num_workers=8)
dataloader_test = torch.utils.data.DataLoader(image_datasets_test, batch_size=4,
                                             shuffle=True, num_workers=8)
# dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
#                                              shuffle=True, num_workers=8)
#                for x in ['train', 'val']}
print("Dataloader size: ", len(dataloader_val))
dataset_sizes_train = len(image_datasets_train)
print(dataset_sizes_train)
dataset_sizes_val = len(image_datasets_val)
print(dataset_sizes_val)
# class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
path_sum = 0
# for i, data in enumerate(dataloader_val):
#     images,labels,paths = data
# #     print( paths[3])
#     if paths[0]: 
#         path_sum+=1
#         print("path 0", paths[0])
#     if paths[1]:
#         path_sum+=1
#         print("path 1", paths[1])
        
#     try:
#         if paths[2]:
#             path_sum+=1
#             print("path 2", paths[2])
            
#     except IndexError:
#         pass
    
#     try:
#         if paths[3]:
#             path_sum+=1
#             print("path 3", paths[3])
            
#     except IndexError:
#         pass
# print("done")
# print(path_sum)
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


# # # Get a batch of training data
inputs, classes, paths = next(iter(dataloader_train))
print(inputs)
# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

imshow(out, title=[class_names[x] for x in classes])


def train_model(model, criterion, optimizer, scheduler, num_epochs):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            if phase == "train":
                
                for inputs, labels, paths in dataloader_train:
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)

                        # backward + optimize only if in training phase
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
            
            
            if phase == "val":
                
                for inputs, labels, paths in dataloader_val:
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)

                        # backward + optimize only if in training phase
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
            
            if phase == 'train':
                scheduler.step()
            if phase == 'train':
                epoch_loss = running_loss / dataset_sizes_train
                epoch_acc = running_corrects.double() / dataset_sizes_train
            if phase == 'val':
                epoch_loss = running_loss / dataset_sizes_val
                epoch_acc = running_corrects.double() / dataset_sizes_val
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    torch.save(model, PATH)
    return model



#Whole Covnet        
# model_ft = models.resnet18(pretrained=True)
# num_ftrs = model_ft.fc.in_features
# # Here the size of each output sample is set to 2.
# # Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
# model_ft.fc = nn.Linear(num_ftrs, 2)

# model_ft = model_ft.to(device)

# criterion = nn.CrossEntropyLoss()

# # Observe that all parameters are being optimized
# optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# # Decay LR by a factor of 0.1 every 7 epochs
# exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

# model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
#                        num_epochs=25)
# visualize_model(model_ft)

#final layer ########################

model_conv = torchvision.models.resnet18(pretrained=True)
for param in model_conv.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 2)

model_conv = model_conv.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized as
# opposed to before.
optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)


model_conv = train_model(model_conv, criterion, optimizer_conv,
                         exp_lr_scheduler, num_epochs=25)



In [ ]:
def visualize_model(model, dataloader):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()
    path_sum = 0
    with torch.no_grad():
        for i, data in enumerate(dataloader):
            inputs, labels, filenames = data
            inputs = inputs.to(device)
            labels = labels.to(device)
              

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
        
#             print(preds)
            if filenames[0]: 
                path_sum+=1
#                 print("path 0", filenames[0])
#                 print("prediction: ", class_names[preds[0]])
                temp = filenames[0].split("/")[-1].split(".")[0].split('_')[0]
#                 dict_key = temp[0]+"_"+temp[1]+"_"+temp[2]+"_"+temp[3]
                dict_key = temp
#                 print(dict_key)
                if dict_key in wsi_all:
                    if class_names[preds[0]] == "Has_DKD":
                        wsi_dict[dict_key]["Has_DKD"] += 1
                    elif class_names[preds[0]] == "Healthy":
                        wsi_dict[dict_key]["Healthy"] += 1
            try:
                if filenames[1]:
                    path_sum+=1
    #                 print("path 1", filenames[1])
    #                 print("prediction: ", class_names[preds[1]])
                    temp = filenames[1].split("/")[-1].split(".")[0].split('_')[0]
    #                 dict_key = temp[0]+"_"+temp[1]+"_"+temp[2]+"_"+temp[3]
                    dict_key = temp
    #                 print(dict_key)                
                    if dict_key in wsi_all:
                        if class_names[preds[1]] == "Has_DKD":
                            wsi_dict[dict_key]["Has_DKD"] += 1
                        elif class_names[preds[1]] == "Healthy":
                            wsi_dict[dict_key]["Healthy"] += 1
    #             print(filenames[2])
            except IndexError:
                pass
            try:
                if filenames[2]:
                    path_sum+=1
#                     print("path 2", filenames[2])
#                     print("prediction: ", class_names[preds[2]])
                    temp = filenames[2].split("/")[-1].split(".")[0].split('_')[0]
                    dict_key = temp
#                     dict_key = temp[0]+"_"+temp[1]+"_"+temp[2]+"_"+temp[3]
#                     print(dict_key)                    
                    if dict_key in wsi_all:
                        if class_names[preds[2]] == "Has_DKD":
                            wsi_dict[dict_key]["Has_DKD"] += 1
                        elif class_names[preds[2]] == "Healthy":
                            wsi_dict[dict_key]["Healthy"] += 1
            except IndexError:
                pass

            try:
                if filenames[3]:
                    path_sum+=1
#                     print("path 3", filenames[3])
#                     print("prediction: ", class_names[preds[3]])
                    temp = filenames[3].split("/")[-1].split(".")[0].split('_')[0]
                    dict_key = temp
#                     dict_key = temp[0]+"_"+temp[1]+"_"+temp[2]+"_"+temp[3]
#                     print(dict_key)
                    if dict_key in wsi_all:
                        if class_names[preds[3]] == "Has_DKD":
                            wsi_dict[dict_key]["Has_DKD"] += 1
                        elif class_names[preds[3]] == "Healthy":
                            wsi_dict[dict_key]["Healthy"] += 1
            except IndexError:
                pass
        

                    
#             print(filenames[0])
#             [print(class_names[x]) for x in preds]
        
#             for j in range(inputs.size()[0]):
#                 images_so_far += 1
#                 ax = plt.subplot(num_images//2, 2, images_so_far)
#                 ax.axis('off')
#                 ax.set_title('predicted: {}'.format(class_names[preds[j]]))
#                 imshow(inputs.cpu().data[j])

#                 if images_so_far == num_images:
#                     model.train(mode=was_training)
#                     return
        model.train(mode=was_training)
model = torch.load(PATH)
model.eval()        
visualize_model(model,dataloader_val)
print("Validation Slide Status: ")
# print(wsi_all)
for k in wsi_all:
#     print(k)
#     print("WSI: ", k, " healthy: ", wsi_dict[k]['Healthy'], "has DKD: ", wsi_dict[k]['Has_DKD'])
    if wsi_dict[k]['Healthy'] > wsi_dict[k]['Has_DKD']:
        print("WSI: ", k, "Status: Healthy" )
    elif wsi_dict[k]['Healthy'] < wsi_dict[k]['Has_DKD']:
        print("WSI: ", k, "Status: Has_DKD" )
plt.ioff()
plt.show()

In [ ]:
model = torch.load(PATH)
model.eval()        
visualize_model(model,dataloader_test)
print("Test Slide Status: ")
# print(wsi_all)
for k in wsi_all:
#     print(k)
#     print("WSI: ", k, " healthy: ", wsi_dict[k]['Healthy'], "has DKD: ", wsi_dict[k]['Has_DKD'])
    if wsi_dict[k]['Healthy'] > wsi_dict[k]['Has_DKD']:
        print("WSI: ", k, "Status: Healthy" )
    elif wsi_dict[k]['Healthy'] < wsi_dict[k]['Has_DKD']:
        print("WSI: ", k, "Status: Has_DKD" )
plt.ioff()
plt.show()


# dataiter = iter(dataloader_test)
# images, labels, paths = dataiter.next()
# images = images.cuda()
# labels = labels.cuda()
# for k in range(4): 
#     imshow(images.cpu().data[k]) 
# print('GroundTruth: ', ' '.join('%5s' % class_names[labels[j]] for j in range(4)))

# outputs = model_conv(images)

# _, predicted = torch.max(outputs, 1)

# print('Predicted: ', ' '.join('%5s' % class_names[predicted[j]]
#                               for j in range(4)))
# correct = 0
# total = 0
# # since we're not training, we don't need to calculate the gradients for our outputs
# with torch.no_grad():
#     path_sum = 0
#     for i, data in enumerate(dataloader_test):
#         inputs, labels, filenames = data
#         images = images.cuda() 
#         labels = labels.cuda()
   
#         # calculate outputs by running images through the network
#         outputs = model_conv(images)
#         # the class with the highest energy is what we choose as prediction
#         _, preds = torch.max(outputs.data, 1)
#         total += labels.size(0)

        
#         if filenames[0]: 
#             path_sum+=1
#             print("path 0", filenames[0])
#             print("prediction: ", class_names[preds[0]])
#             temp = filenames[0].split("/")[-1].split(".")[0].split('_')[0]
# #                 dict_key = temp[0]+"_"+temp[1]+"_"+temp[2]+"_"+temp[3]
#             dict_key = temp
# #             print(dict_key)
#             if dict_key in wsi_all:
#                 if class_names[preds[0]] == "Has_DKD":
#                     wsi_dict[dict_key]["Has_DKD"] += 1
#                 elif class_names[preds[0]] == "Healthy":
#                     wsi_dict[dict_key]["Healthy"] += 1
#                 elif class_names[preds[0]] == "CKD_noDM":
#                     wsi_dict[dict_key]["CKD_noDM"] += 1   
#         if filenames[1]:
#             path_sum+=1
#             print("path 1", filenames[1])
#             print("prediction: ", class_names[preds[1]])
#             temp = filenames[1].split("/")[-1].split(".")[0].split('_')[0]
# #                 dict_key = temp[0]+"_"+temp[1]+"_"+temp[2]+"_"+temp[3]
#             dict_key = temp
# #             print(dict_key)                
#             if dict_key in wsi_all:
#                 if class_names[preds[1]] == "Has_DKD":
#                     wsi_dict[dict_key]["Has_DKD"] += 1
#                 elif class_names[preds[1]] == "Healthy":
#                     wsi_dict[dict_key]["Healthy"] += 1
#                 elif class_names[preds[0]] == "CKD_noDM":
#                     wsi_dict[dict_key]["CKD_noDM"] += 1   
# #             print(filenames[2])
#         try:
#             if filenames[2]:
#                 path_sum+=1
#                 print("path 2", filenames[2])
#                 print("prediction: ", class_names[preds[2]])
#                 temp = filenames[2].split("/")[-1].split(".")[0].split('_')[0]
#                 dict_key = temp
# #                     dict_key = temp[0]+"_"+temp[1]+"_"+temp[2]+"_"+temp[3]
# #                 print(dict_key)                    
#                 if dict_key in wsi_all:
#                     if class_names[preds[2]] == "Has_DKD":
#                         wsi_dict[dict_key]["Has_DKD"] += 1
#                     elif class_names[preds[2]] == "Healthy":
#                         wsi_dict[dict_key]["Healthy"] += 1
#                     elif class_names[preds[0]] == "CKD_noDM":
#                         wsi_dict[dict_key]["CKD_noDM"] += 1   
#         except IndexError:
#             pass

#         try:
#             if filenames[3]:
#                 path_sum+=1
#                 print("path 3", filenames[3])
#                 print("prediction: ", class_names[preds[3]])
#                 temp = filenames[3].split("/")[-1].split(".")[0].split('_')[0]
#                 dict_key = temp
# #                     dict_key = temp[0]+"_"+temp[1]+"_"+temp[2]+"_"+temp[3]
# #                 print(dict_key)
#                 if dict_key in wsi_all:
#                     if class_names[preds[3]] == "Has_DKD":
#                         wsi_dict[dict_key]["Has_DKD"] += 1
#                     elif class_names[preds[3]] == "Healthy":
#                         wsi_dict[dict_key]["Healthy"] += 1
#                     elif class_names[preds[0]] == "CKD_noDM":
#                         wsi_dict[dict_key]["CKD_noDM"] += 1   
#         except IndexError:
#             pass

# # print('Accuracy of the network on the 10000 test images: %d %%' % (
# #     100 * correct / total))


# # prepare to count predictions for each class
# correct_pred = {classname: 0 for classname in class_names}
# total_pred = {classname: 0 for classname in class_names}

# # again no gradients needed
# with torch.no_grad():
#     for i,data in enumerate(dataloader_test):
#         images, labels, paths = data
#         images = images.cuda()
#         labels = labels.cuda()
#         outputs = model_conv(images)
#         _, predictions = torch.max(outputs, 1)
#         # collect the correct predictions for each class
#         for label, prediction in zip(labels, predictions):
#             if label == prediction:
#                 correct_pred[class_names[label]] += 1
#             total_pred[class_names[label]] += 1


# # # print accuracy for each class
# for classname, correct_count in correct_pred.items():
#     accuracy = 100 * float(correct_count) / total_pred[classname]
#     print("Accuracy for class {:5s} is: {:.1f} %".format(classname,
#                                                    accuracy))
    
# print("Testing & Validation Slide Status: ")
# # print(wsi_all)
# for k in wsi_all:
# #     print(k)
# #     print("WSI: ", k, " healthy: ", wsi_dict[k]['Healthy'], "has DKD: ", wsi_dict[k]['Has_DKD'])
#     if wsi_dict[k]['Healthy'] > wsi_dict[k]['Has_DKD']:
#         print("WSI: ", k, "Status: Healthy" )
#     elif wsi_dict[k]['Healthy'] < wsi_dict[k]['Has_DKD']:
#         print("WSI: ", k, "Status: Has_DKD" )